In [3]:
import pandas as pd
import math
import numpy as np

In [4]:
report = pd.read_csv('https://raw.githubusercontent.com/HiddenS1/Newsroom/main/data/Index_Crimes_by_County_and_Agency__Beginning_1990%20(1).csv')
NY_part = pd.read_csv("https://raw.githubusercontent.com/HiddenS1/Newsroom/main/data/NY_part.csv")

In [40]:
report.head()

,County,Agency,Year,Months Reported,Index Total,Violent Total,Murder,Rape,Robbery,Aggravated Assault,Property Total,Burglary,Larceny,Motor Vehicle Theft,Region
0,Albany,Albany City PD,2021,12.0,3837.0,945.0,24.0,60.0,179.0,682.0,2892.0,368.0,2170.0,354.0,Non-New York City
1,Albany,Albany County Sheriff,2021,12.0,73.0,18.0,0.0,0.0,1.0,17.0,55.0,9.0,41.0,5.0,Non-New York City
2,Albany,Albany County State Police,2021,12.0,125.0,43.0,0.0,27.0,1.0,15.0,82.0,7.0,66.0,9.0,Non-New York City
3,Albany,Altamont Vg PD,2021,12.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,Non-New York City
4,Albany,Bethlehem Town PD,2021,12.0,329.0,22.0,1.0,5.0,4.0,12.0,307.0,27.0,269.0,11.0,Non-New York City


In [58]:
NY_part.sample(10)

,agency_type,FIPS,agency_name_full,population_21,months_reported_in_21,is_nibrs_2022,months_reported_in_22,violent_crime_2020,property_crime_2020,murder_2020,total_crime_2020,longitude,latitude,Info
95,County,36029.0,erie county sheriff's office,126954,0,0,0,52.0,575.0,2.0,627.0,-78.778192,42.752759,NaN
342,City,36089.0,potsdam village police department,8943,8,1,9,2.0,45.0,0.0,47.0,-75.074311,44.488112,NaN
162,City,36045.0,black river village pd black river,1229,0,0,0,0.0,3.0,0.0,3.0,-76.053514,43.995371,NaN
532,State Police,36083.0,sp: rensselaer county,0,0,0,0,44.0,272.0,0.0,316.0,-73.513845,42.710421,NaN
122,City,36033.0,tupper lake village police department,3420,0,0,0,1.0,25.0,0.0,26.0,-74.310670,44.594376,NaN
370,City,36101.0,hornell police department,8272,0,0,0,0.0,0.0,0.0,0.0,-77.385525,42.266725,NaN
496,State Police,36007.0,sp: broome county,0,0,0,0,74.0,389.0,1.0,463.0,-75.830283,42.161977,NaN
118,City,36031.0,lake placid village police department,2339,0,0,0,7.0,16.0,0.0,23.0,-73.777573,44.108971,NaN
477,City,36119.0,harrison town police department,29110,0,0,0,6.0,126.0,0.0,132.0,-73.745912,41.152770,NaN
478,City,36119.0,lewisboro town police department,12533,0,0,0,0.0,0.0,0.0,0.0,-73.745912,41.152770,NaN


In [286]:
NY_part.agency_type.unique()

array(['City', 'County', 'Other State Agency', 'Other',
       'University or College', 'State Police'], dtype=object)

In [5]:
# In order to merge the two tables on agency names
# we categorize them by agency type
city_words = ['city pd', 'town pd', 'vg pd']
county_word = 'county sheriff'
sp_word = 'state police'

In [6]:
is_city = [any(word in agency.lower() for word in city_words) for agency in report.Agency]
city = report.loc[is_city]

In [7]:
city_NY = NY_part.loc[NY_part.agency_type=='City','agency_name_full'].unique().tolist()

In [8]:
city_report = city.Agency.unique().tolist()

In [9]:
from thefuzz import fuzz, process

In [10]:
city_connect = {}
for i in range(len(city_NY)):
    best_match = process.extractOne(city_NY[i], city_report, scorer=fuzz.ratio)
    city_connect.update({city_NY[i]: best_match[0]})

In [51]:
city_connect

{'albany police department': 'Albany City PD',
 'cohoes police department': 'Cohoes City PD',
 'watervliet police department': 'Watervliet City PD',
 'green island vlg police dept': 'Green Island Vg PD',
 'altamont village police department': 'Altamont Vg PD',
 'menands village police department': 'Menands Vg PD',
 'bethlehem town police department': 'Bethlehem Town PD',
 'guilderland town police department': 'Guilderland Town PD',
 'colonie town police department': 'Colonie Town PD',
 'coeymans town police department': 'Coeymans Town PD',
 'alfred village police department': 'Alfred Vg PD',
 'andover village police department': 'Andover Vg PD',
 'angelica vlg police dept': 'Angelica Vg PD',
 'bolivar village police department': 'Bolivar Vg PD',
 'cuba town police department': 'Cuba Town PD',
 'wellsville village police department': 'Wellsville Vg PD',
 'friendship town police dept': 'Friendship Town PD',
 'belmont village police department': 'Belmont Vg PD',
 'independence town police

In [11]:
city_connect['lakewood-busti police department'] = 'Lakewood-Busti PD'
city_connect['east aurora village pd east aurora'] = 'East Aurora-Town Of Aurora PD'
city_connect['port washington police district'] = 'Port Washington Police District'
city_connect['troy police department'] = 'Troy City PD'
city_connect['south nyack - grand view village police department'] = 'South Nyack-Grand View PD'
city_connect.pop('columbia greene humane society')
city_connect.pop('theresa village pd theresa')
city_connect.pop('new york city police department')
city_connect.pop('barker village police department')

'Waterville Vg PD'

In [12]:
county_NY = NY_part.loc[NY_part.agency_type=='County','agency_name_full'].unique().tolist()

In [13]:
county_report = list(set([agency for agency in report.Agency if county_word in agency.lower()]))

In [14]:
county_connect = {}
for i in range(len(county_NY)):
    best_match = process.extractOne(county_NY[i], county_report, scorer=fuzz.ratio)
    county_connect.update({county_NY[i]: best_match[0]})

In [74]:
county_connect

{"albany county sheriff's office": 'Albany County Sheriff',
 "allegany county sheriff's office": 'Allegany County Sheriff',
 "broome county sheriff's office": 'Broome County Sheriff',
 "cattaraugus county sheriff's office": 'Cattaraugus County Sheriff',
 "cayuga county sheriff's office": 'Cayuga County Sheriff',
 "chautauqua county sheriff's office": 'Chautauqua County Sheriff',
 "chemung county sheriff's office": 'Chemung County Sheriff',
 "chenango county sheriff's office": 'Chenango County Sheriff',
 "clinton county sheriff's office": 'Clinton County Sheriff',
 "columbia county sheriff's office": 'Columbia County Sheriff',
 "cortland county sheriff's office": 'Cortland County Sheriff',
 "delaware county sheriff's office": 'Delaware County Sheriff',
 "dutchess county sheriff's office": 'Dutchess County Sheriff',
 "erie county sheriff's office": 'Erie County Sheriff',
 "essex county sheriff's office": 'Essex County Sheriff',
 "franklin county sheriff's office": 'Franklin County Sherif

In [15]:
county_connect['nassau county police department'] = 'Nassau County PD'
county_connect['suffolk county police department'] = 'Suffolk County PD'
county_connect['westchester county department of public safety'] = 'Westchester Co Dept Pub Safety'

In [16]:
city_connect.update(county_connect)

In [17]:
sp_NY = NY_part.loc[NY_part.agency_type=='State Police','agency_name_full'].unique().tolist()
sp_report = list(set([agency for agency in report.Agency if sp_word in agency.lower()]))

In [18]:
sp_connect = {}
for i in range(len(sp_NY)):
    best_match = process.extractOne(sp_NY[i], sp_report, scorer=fuzz.ratio)
    sp_connect.update({sp_NY[i]: best_match[0]})

In [349]:
sp_connect

{'sp: albany county': 'Albany County State Police',
 'sp: allegany county': 'Allegany County State Police',
 'sp: broome county': 'Broome County State Police',
 'sp: cattaraugus county': 'Cattaraugus County State Police',
 'sp: cayuga county': 'Cayuga County State Police',
 'sp: chautauqua county': 'Chautauqua County State Police',
 'sp: chemung county': 'Chemung County State Police',
 'sp: chenango county': 'Chenango County State Police',
 'sp: clinton county': 'Clinton County State Police',
 'sp: columbia county': 'Columbia County State Police',
 'sp: cortland county': 'Cortland County State Police',
 'sp: delaware county': 'Delaware County State Police',
 'sp: dutchess county': 'Dutchess County State Police',
 'sp: erie county': 'Erie County State Police',
 'sp: essex county': 'Essex County State Police',
 'sp: franklin county': 'Franklin County State Police',
 'sp: fulton county': 'Fulton County State Police',
 'sp: genesee county': 'Genesee County State Police',
 'sp: greene count

In [19]:
sp_connect.pop('sp: new york county')

'Ulster County State Police'

In [20]:
city_connect.update(sp_connect)

In [21]:
report_update = list(city_connect.values())

In [22]:
SUNY = report.loc[['SUNY' in agency for agency in report['Agency']],'Agency'].unique().tolist()
SUNY.remove('SUNY College at Utica/Rome') 
SUNY.remove('SUNY College At Utica/Rome')
SUNY.remove('SUNY - Albany (Plaza)')
SUNY.remove('SUNY - Buffalo at Amherst')

In [23]:
def uni(agency):
    if agency=='SUNY College-Env. Science and Forestry':
        name = 'Environmental Science'
    if '-' in agency:
        name = agency.split('- ')[-1]
    elif 'at ' in agency:
        name = agency.split('at ')[-1]
    else:
        name = agency.split('At ')[-1]
    if name=='Upstate Medical Center':
        name = 'Upstate Medical'
    return 'SUNY ' + name

In [24]:
college = report.loc[[agency in SUNY for agency in report.Agency]]
college['Agency'] = college['Agency'].apply(uni)

C:\Users\28047\AppData\Local\Temp\ipykernel_37064\1701706639.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  college['Agency'] = college['Agency'].apply(uni)


In [25]:
report.loc[[agency in SUNY for agency in report.Agency]]['Agency'] = report.loc[[agency in SUNY for agency in report.Agency]]['Agency'].apply(uni)

C:\Users\28047\AppData\Local\Temp\ipykernel_37064\4192484874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report.loc[[agency in SUNY for agency in report.Agency]]['Agency'] = report.loc[[agency in SUNY for agency in report.Agency]]['Agency'].apply(uni)


In [34]:
#report.to_csv('D:/temp/Columbia/newsroom/Marshall/corrected_index_crime.csv', index=False)

In [35]:
report = report.loc[[agency in report_update for agency in report.Agency]]

In [36]:
report = pd.concat([report, college])

In [37]:
nan_report = [math.isnan(month) for month in report['Months Reported']]
report_21 = report.loc[(np.logical_not(nan_report)) & (report.Year < 2021)]

In [318]:
report_21.sample(5)

,County,Agency,Year,Months Reported,Index Total,Violent Total,Murder,Rape,Robbery,Aggravated Assault,Property Total,Burglary,Larceny,Motor Vehicle Theft,Region
1871,Westchester,Yorktown Town PD,2019,12.0,222.0,8.0,0.0,1.0,3.0,4.0,214.0,20.0,186.0,8.0,Non-New York City
4569,Cortland,Cortland City PD,2014,12.0,426.0,27.0,2.0,9.0,3.0,13.0,399.0,86.0,298.0,15.0,Non-New York City
10167,Nassau,Rockville Centre Vg PD,2006,12.0,325.0,21.0,0.0,1.0,17.0,3.0,304.0,42.0,240.0,22.0,Non-New York City
9032,Wayne,Wayne County Sheriff,2008,12.0,673.0,51.0,0.0,7.0,7.0,37.0,622.0,150.0,445.0,27.0,Non-New York City
12036,Westchester,Ossining Vg PD,2004,12.0,337.0,71.0,0.0,8.0,38.0,25.0,266.0,33.0,190.0,43.0,Non-New York City


In [38]:
report_in_21 = report.loc[(np.logical_not(nan_report)) & (report.Year == 2021),['Agency','Months Reported']]

In [39]:
report_by_agency = report_21.groupby(['County','Agency'])['Months Reported'].agg(['mean', 'count']).reset_index().rename(columns={'mean': 'Historical mean','count':'Historical count'})

In [360]:
report_by_agency.sample(20)

,County,Agency,Historical mean,Historical count
314,Orange,Highland Falls Vg PD,11.631579,19
127,Erie,Hamburg Vg PD,12.000000,19
459,Tioga,Tioga County Sheriff,12.000000,19
57,Chautauqua,Westfield Vg PD,12.000000,19
425,Steuben,Painted Post Vg PD,11.631579,19
488,Warren,Warren County State Police,12.000000,19
494,Washington,Washington County Sheriff,12.000000,19
129,Erie,Lackawanna City PD,12.000000,19
538,Westchester,Scarsdale Vg PD,12.000000,19
350,Otsego,Oneonta City PD,11.947368,19


In [40]:
NY_county = NY_part.loc[:,['agency_type','population_21','months_reported_in_21','Info',
                           'months_reported_in_22','agency_name_full','violent_crime_2020',
                           'property_crime_2020','murder_2020','total_crime_2020','longitude','latitude']]

In [41]:
NY_county1 = NY_county.loc[NY_county.agency_type!='University or College']
NY_county2 = NY_county.loc[NY_county.agency_type=='University or College']
NY_county1['Agency'] = NY_county1['agency_name_full'].apply(lambda x: city_connect[x] if city_connect.get(x) else 'nan')
NY_county2['Agency'] = NY_county2['Info']
NY_county = pd.concat([NY_county1,NY_county2])

C:\Users\28047\AppData\Local\Temp\ipykernel_37064\1988566305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NY_county1['Agency'] = NY_county1['agency_name_full'].apply(lambda x: city_connect[x] if city_connect.get(x) else 'nan')
C:\Users\28047\AppData\Local\Temp\ipykernel_37064\1988566305.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NY_county2['Agency'] = NY_county2['Info']


In [43]:
NY_county.sample(5)

,agency_type,population_21,months_reported_in_21,Info,months_reported_in_22,agency_name_full,violent_crime_2020,property_crime_2020,murder_2020,total_crime_2020,longitude,latitude,Agency
128,City,1048,0,NaN,0,northville village police department,0.0,3.0,0.0,3.0,-74.423678,43.115609,Northville Vg PD
15,City,812,0,NaN,0,angelica vlg police dept,0.0,0.0,0.0,0.0,-78.026115,42.247871,Angelica Vg PD
434,City,1708,0,NaN,0,greenwich village police department,0.0,11.0,0.0,11.0,-73.439428,43.312377,Greenwich Vg PD
65,County,43015,0,NaN,0,columbia county sheriff's office,26.0,254.0,0.0,280.0,-73.626803,42.247728,Columbia County Sheriff
153,City,1706,0,NaN,0,adams vlg police dept,0.0,3.0,0.0,3.0,-76.053514,43.995371,Adams Vg PD


In [45]:
all_agency = pd.merge(NY_county, report_by_agency, on='Agency', how='left')

In [46]:
all_agency = pd.merge(report_in_21, all_agency, on='Agency', how='right')

In [47]:
all_agency.rename(columns={'Months Reported': 'Index Crime Months','months_reported_in_21':'NIBRS Months_21',
                           'months_reported_in_22':'NIBRS Months_22'},inplace=True)

In [48]:
all_agency.drop(columns=['Info'], inplace=True)

In [51]:
# Check the historical data of these agencies
all_agency.sample(10)

,Agency,Index Crime Months,agency_type,population_21,NIBRS Months_21,NIBRS Months_22,agency_name_full,violent_crime_2020,property_crime_2020,murder_2020,total_crime_2020,longitude,latitude,County,Historical mean,Historical count
418,Lloyd Town PD,12.0,City,10484,10,12,lloyd town police department,6.0,81.0,0.0,87.0,-74.265458,41.947216,Ulster,11.631579,19.0
522,Niagara County State Police,12.0,State Police,0,0,0,sp: niagara county,38.0,214.0,1.0,252.0,-78.792142,43.456731,Niagara,12.000000,19.0
507,Essex County State Police,12.0,State Police,0,0,0,sp: essex county,44.0,206.0,0.0,250.0,-73.777573,44.108971,Essex,12.000000,19.0
431,Washington County Sheriff,12.0,County,41826,0,0,washington county sheriff's office,21.0,73.0,0.0,94.0,-73.439428,43.312377,Washington,12.000000,19.0
213,Centre Island Vg PD,NaN,City,409,0,0,centre island village police department,0.0,1.0,0.0,1.0,-73.589419,40.729612,Nassau,11.777778,18.0
309,Oneonta City PD,12.0,City,13904,11,12,oneonta police department,19.0,175.0,0.0,194.0,-75.028841,42.629776,Otsego,11.947368,19.0
360,Schoharie Vg PD,12.0,City,806,0,0,schoharie village pd schoharie,0.0,4.0,0.0,4.0,-74.438172,42.591294,Schoharie,12.000000,13.0
546,Washington County State Police,12.0,State Police,0,0,0,sp: washington county,33.0,92.0,0.0,125.0,-73.439428,43.312377,Washington,12.000000,19.0
76,Hancock Vg PD,12.0,City,923,0,0,hancock village police department,1.0,8.0,0.0,9.0,-74.966728,42.193986,Delaware,12.000000,17.0
512,Hamilton County State Police,12.0,State Police,0,0,0,sp: hamilton county,6.0,25.0,0.0,31.0,-74.502456,43.657879,Hamilton,12.000000,19.0


In [52]:
#all_agency.to_csv('D:/temp/Columbia/newsroom/Marshall/all_agency_NY.csv', index=False)

In [53]:
county_info = all_agency.groupby('County').agg({'population_21': 'sum', 'violent_crime_2020': 'sum', 
                                  'property_crime_2020': 'sum', 'murder_2020': 'sum', 
                                  'total_crime_2020': 'sum', 'NIBRS Months_21': 'mean', 
                                  'NIBRS Months_22': 'mean', 'Index Crime Months': 'mean', 'Historical mean': 'mean'}).reset_index()

In [54]:
county_info['count'] = all_agency.groupby('County')['population_21'].count().values

In [372]:
# Participation level by county in 2021 and 2022
county_info

,County,population_21,violent_crime_2020,property_crime_2020,murder_2020,total_crime_2020,NIBRS Months_21,NIBRS Months_22,Index Crime Months,Historical mean,count
0,Albany,303723.0,1091.0,6204.0,17.0,7295.0,1.714286,3.428571,11.583333,11.916667,14
1,Allegany,45491.0,71.0,320.0,3.0,391.0,0.000000,0.000000,11.818182,11.493092,12
2,Broome,175763.0,559.0,4080.0,4.0,4639.0,6.857143,6.857143,10.857143,11.837510,7
3,Cattaraugus,73627.0,123.0,718.0,1.0,841.0,2.400000,2.400000,12.000000,11.788259,10
4,Cayuga,75732.0,117.0,635.0,1.0,752.0,2.000000,3.666667,12.000000,11.958333,6
5,Chautauqua,125262.0,313.0,2307.0,1.0,2620.0,10.300000,9.700000,12.000000,12.000000,10
6,Chemung,82347.0,173.0,1336.0,3.0,1509.0,0.000000,0.000000,12.000000,11.964912,6
7,Chenango,46567.0,109.0,593.0,0.0,702.0,0.000000,1.000000,10.833333,11.750363,9
8,Clinton,79798.0,122.0,1008.0,1.0,1130.0,0.000000,0.000000,12.000000,12.000000,4
9,Columbia,56117.0,86.0,597.0,0.0,683.0,2.000000,1.333333,11.000000,11.000000,6


In [373]:
#county_info.to_csv('D:/temp/Columbia/newsroom/Marshall/county_info.csv',index=False)